In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
from skimage.draw import circle, line
from skimage.measure import regionprops
import tqdm
import matplotlib.image as mpimg
from scipy.spatial import distance as dist

In [2]:
def atoi(text):
   return int(text) if text.isdigit() else text
def natural_keys(text):
   return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
image_list = []
for filename in sorted(glob.glob('track_cells/*.tiff'), key=natural_keys): 
    im=cv2.imread(filename)
   
    image_list.append(im)

In [4]:
def return_cord(i_0):    
    cx = []
    cy = []

    shifted = cv2.pyrMeanShiftFiltering(i_0, 10, 51)
    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=9, labels=thresh)
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thresh)



    for label in np.unique(labels):
        if label == 0:
            continue




        mask = np.zeros(gray.shape, dtype='uint8')
        mask[labels == label] = 255
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        ((x, y), r) = cv2.minEnclosingCircle(c)



        cx.append(int(x))
        cy.append(int(y))
        
        l1 = np.column_stack((cx,cy))
        

    return l1, cx, cy

In [5]:
i_0 = image_list[0]

i_1 = image_list[1]

i_2 = image_list[2]

i_3 = image_list[3]

i_4 = image_list[4]

i_5 = image_list[5]

In [6]:
cor_0,x0,y0 = return_cord(i_0)
cor_1,x1,y1 = return_cord(i_1)
cor_2,x2,y2 = return_cord(i_2)
cor_3,x3,y3 = return_cord(i_3)
cor_4,x4,y4 = return_cord(i_4)
cor_5,x5,y5 = return_cord(i_5)

In [7]:
def eucl_dist_matrix(cor_t, cor_t1):
    
    d = dist.cdist(cor_t, cor_t1)
    rows = d.min(axis=1).argsort()
    cols = d.argmin(axis=1)[rows]
    
    list_id_index = (list(zip(rows, cols)))
    list_id_index = sorted(list_id_index)
    
    
    
    
    return rows, cols, list_id_index ,d 

In [46]:
rows1, cols1, matrix1, d1 = eucl_dist_matrix(correct_2, cor_3)

d1.shape

(92, 103)

In [47]:
row1, col1 = zip(*matrix1)

In [48]:
#Check for unused columns
def check_unused_col(cor_t_1, col_t_1):
    
    unused_col = list(range(0, len(cor_t_1)))
    unused = set(col_t_1)
    idx_unused_col = [x for x in unused_col if x not in unused]
    
    return idx_unused_col


In [49]:
idx_unused_col = check_unused_col(cor_1, col1)
len(idx_unused_col)

12

In [50]:
#Check for duplicates
def check_duplicates_col(col_t1):
    duplicates2 = []
    for item in col_t1:
        if col_t1.count(item) > 1:
            duplicates2.append(item)
    dup_set2 = set(duplicates2)
    
    return dup_set2

In [51]:
duplicate = check_duplicates_col(col1)
len(duplicate)

1

In [52]:
# IF LEN OF DUPLICATES = 0, DO NOT DO THIS

d = []
r = []
c = []

for ix, (a, b) in enumerate(zip(row1, col1)):
    for i in duplicate: 
        if b == i:
            d.append(d1[a][b])
            r. append(a)
            c.append(b)

In [53]:
w = (list(zip(r,c,d)))
w.sort(key = lambda t: t[1])


In [54]:
temp = {}
for row, col, di in w:
    if col not in temp: # we see this key for the first time
        temp[col] = (row, col, di)
    else:
        if temp[col][2] > di: 
            temp[col] = (row, col, di)

result = temp.values()

result = list(result)



In [55]:
l2set = set(result)
l3 = [x for x in w if x not in l2set]


In [56]:
rem_row, rem_col, rem_dit = zip(*l3)
rem_cor = (list(zip(rem_row,rem_col)))


In [57]:
rem_cor = set(rem_cor)
keep_cor = [x for x in matrix1 if x not in rem_cor]
keep_cor # cordinates that have to be removed
keep_cor.sort(key = lambda t: t[1])


In [59]:
new_rowt1, new_colt1 = zip(*keep_cor)
large = len(new_rowt1)
large
lst = list(range(0,(large+1)))
ix = set(new_rowt1)
    
m = [x for x in lst if x not in ix]
    
centroid_t2 = (list(zip(x_2,y_2, id_cell)))
    
real_centroid2 = [i for j, i in enumerate(centroid_t2) if j not in m]
    
x_2, y_2, id_cell2= zip(*real_centroid2)
    
correct_2 = np.column_stack((x_2,y_2))



In [62]:
# Unused colums

miss_idx = idx_unused_col
miss_idx
centroid_t3 = (list(zip(x3, y3)))
real_centroid3 = [i for j, i in enumerate(centroid_t3) if j not in miss_idx]

In [63]:
rows1, cols1, matrix1, d1 = eucl_dist_matrix(correct_2, real_centroid3)

d1.shape

(91, 91)

In [64]:
row1, col1 = zip(*matrix1)

correct_3= []

for ix, (i,j) in enumerate(zip(row1, col1)):
    correct_3.insert(ix, real_centroid3[j])
    


In [65]:
x_3, y_3 = zip(*correct_3)

In [25]:
def get_image(x, y, id_cell, image, number_image):
    
    df2 = pd.DataFrame(list(zip(x, y, id_cell)), 
               columns =['row', 'col', 'id_cell']) 


    x = []
    y = []
    corrdinate = []
    for i in range(len(df2)):
        x.append(df2.iloc[i,0])
        y.append(df2.iloc[i,1])
        corrdinate.append(df2.iloc[i,2])



    for idx, i in enumerate(x):



        cv2.circle(image, (x[idx], y[idx]), 3, (0, 255, 255), -1)
        cv2.putText(image, "{0}".format(corrdinate[idx]), (x[idx] - 20, y[idx] - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imwrite('img_{0}.png'.format(number_image), image)

In [66]:
get_image(x_3, y_3, id_cell2, i_3, '3')